In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")


d:\Apps\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 42.0/42.0 [00:00<?, ?B/s]
d:\Apps\Python\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tigra\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-ru. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrat

In [2]:

# Пути к файлам с данными
train_data_file = "train_data.txt"
val_data_file = "val_data.txt"

# Функция для чтения данных из файла
def read_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.readlines()
    return data

# Чтение данных для обучения и валидации
train_data = read_data(train_data_file)
val_data = read_data(val_data_file)

# Преобразование данных в формат, понимаемый Seq2SeqDataset
def process_data(data):
    processed_data = []
    for line in data:
        source_text, target_text = line.strip().split(":")
        processed_data.append({"source_text": source_text.strip(), "target_text": target_text.strip()})
    return processed_data

train_processed_data = process_data(train_data)
val_processed_data = process_data(val_data)

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_source_length, max_target_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source_text = self.data[index]["source_text"]
        target_text = self.data[index]["target_text"]

        # Токенизация и добавление специальных токенов
        source_inputs = self.tokenizer.encode_plus(source_text, max_length=self.max_source_length, padding="max_length", truncation=True, return_tensors="pt")
        target_inputs = self.tokenizer.encode_plus(target_text, max_length=self.max_target_length, padding="max_length", truncation=True, return_tensors="pt")

        source_ids = source_inputs["input_ids"].squeeze()
        source_mask = source_inputs["attention_mask"].squeeze()
        target_ids = target_inputs["input_ids"].squeeze()

        return {
            "input_ids": source_ids,
            "attention_mask": source_mask,
            "labels": target_ids,
        }

# Создание датасетов
train_dataset = CustomDataset(train_processed_data, tokenizer, max_source_length=128, max_target_length=128)
val_dataset = CustomDataset(val_processed_data, tokenizer, max_source_length=128, max_target_length=128)




In [7]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps",
    save_steps=10,
    num_train_epochs=10,
    overwrite_output_dir=True,
    # Добавьте любые другие аргументы, необходимые для fine-tuning
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # ваш тренировочный датасет
    eval_dataset=val_dataset,     # ваш валидационный датасет
    tokenizer=tokenizer
)

trainer.train()

d:\Apps\Python\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
 10%|█         | 10/100 [00:20<02:40,  1.79s/it]

{'loss': 0.0369, 'grad_norm': 1.88331139087677, 'learning_rate': 4.5e-05, 'epoch': 1.0}



 10%|█         | 10/100 [00:21<02:40,  1.79s/it]Checkpoint destination directory ./results\checkpoint-10 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}


{'eval_loss': 0.06804201751947403, 'eval_runtime': 0.3696, 'eval_samples_per_second': 5.411, 'eval_steps_per_second': 2.705, 'epoch': 1.0}


 20%|██        | 20/100 [00:42<02:24,  1.81s/it]

...


100%|██████████| 100/100 [03:30<00:00,  1.76s/it]Checkpoint destination directory ./results\checkpoint-100 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62517]], 'forced_eos_token_id': 0}


{'eval_loss': 0.010417242534458637, 'eval_runtime': 0.3582, 'eval_samples_per_second': 5.584, 'eval_steps_per_second': 2.792, 'epoch': 10.0}


100%|██████████| 100/100 [03:32<00:00,  2.12s/it]

{'train_runtime': 212.4354, 'train_samples_per_second': 1.789, 'train_steps_per_second': 0.471, 'train_loss': 0.011130043976008893, 'epoch': 10.0}


TrainOutput(global_step=100, training_loss=0.011130043976008893, metrics={'train_runtime': 212.4354, 'train_samples_per_second': 1.789, 'train_steps_per_second': 0.471, 'train_loss': 0.011130043976008893, 'epoch': 10.0})

In [4]:
def interact_with_model(model, tokenizer, input_text):
    # Предобработка входных данных
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Передача данных в модель
    output_ids = model.generate(input_ids, max_length=128, num_beams=4, early_stopping=True)

    # Декодирование выходных данных
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text


In [12]:
input_text = "плз"
output_text = interact_with_model(model, tokenizer, input_text)
print("Восстановленное сокращение:", output_text)

Восстановленное сокращение: плаз
